In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load model and tokenizer
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Reset the existing chat template before setting up the chat format
tokenizer.chat_template = None

# Now setup the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

finetune_name = "DistressAI"
finetune_tags = ["DistressAI"]

In [ ]:
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
user
Sure! Here's a haiku about programming:

Programming languages flow,
Code runs on the screen,
Syntax rules are simple.


In [ ]:
ds = load_dataset("HuggingFaceTB/smoltalk", "all")
ds


In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  
    per_device_train_batch_size=4, 
    learning_rate=5e-5,  
    logging_steps=10,  
    save_steps=100, 
    evaluation_strategy="steps", 
    eval_steps=50,  
    use_mps_device=(
        True if device == "mps" else False
    ),  
    hub_model_id=finetune_name,  
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

In [ ]:
model_name = "NguyenDuyPhuc/DistressAI"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Test with a clear prompt
prompt = "What should I do to help my families"
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Apply chat template
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)


print("Model response:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))